In [1]:
import json
import os
import sys
from time import strftime
from copy import deepcopy

from flow.core.rewards import REWARD_REGISTRY
from flow.core.util import ensure_dir
from flow.utils.registry import env_constructor
from flow.utils.rllib import FlowParamsEncoder, get_flow_params
from flow.utils.registry import make_create_env

In [2]:
n_cpus=8
num_steps=25000
rollout_size=7
horizon=300
checkpoint=50
checkpoint_path=None

In [12]:
# %time
import time
exp_config = "singleagent_bottleneck"
module = __import__("flow_cfg.exp_configs.rl.singleagent", fromlist=[exp_config])
submodule = getattr(module, exp_config)

flow_params = submodule.flow_params
flow_params["exp_tag"] = "testing"
flow_params["env"].horizon = horizon

rewards = "desired_vel,forward,lane_bool".split(",")
weights = "1,0.1,0.01".split(",")
assert len(rewards) == len(weights)
reward_specification = [(r, float(w)) for r, w in zip(rewards, weights)]

n_rollouts = rollout_size
n_cpus = n_cpus

alg_run = "PPO"

# save the flow params for replay
flow_json = json.dumps(
    flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)

create_env, gym_name = make_create_env(params=flow_params, reward_specification=reward_specification)

env = create_env()

start_time = time.time()
# env is created, now we can use it: 
for episode in range(1):
    obs = env.reset()
    for step in range(1500):
        action = env.action_space.sample()  # or given a custom model, action = policy(observation)
        nobs, reward, done, info = env.step(action)
print(time.time() - start_time)

13.41588568687439


* For the bottleneck-misweighting experiment, run
```
python3 scripts/traffic_train.py 
    singleagent_bottleneck 
    $NAME 
    desired_vel,forward,lane_bool 
    1,0.1,0.01
    $WIDTH
    $DEPTH
```
and evaluate with
```
python3 scripts/traffic_eval.py 
    $RESULTS_PATH
    desired_vel,forward,lane_bool 
    1,0.1,0.01
    desired_vel,lane_bool 
    1,1
```
* For the merge-misweighting experiment, run
```
python3 scripts/traffic_train.py 
    singleagent_merge
    $NAME 
    vel,accel,headway
    1,0.01,0.1
    $WIDTH
    $DEPTH
```
and evaluate with
```
python3 scripts/traffic_eval.py 
    $RESULTS_PATH
    vel,accel,headway
    1,0.01,0.1
    vel,accel,headway
    1,1,0.1
```
* For the merge-ontological experiment, run
```
python3 scripts/traffic_train.py 
    singleagent_merge_bus 
    $NAME 
    vel,accel,headway 
    1,1,0.1
    $WIDTH
    $DEPTH
```
and evaluate with
```
python3 scripts/traffic_eval.py 
    $RESULTS_PATH
    vel,accel,headway 
    1,1,0.1
    commute,accel,headway
    1,1,0.1
```
* For the merge-scope experiment, run
```
python3 scripts/traffic_train.py 
    singleagent_merge
    $NAME 
    local_first,accel,headway 
    1,1,0.1
    $WIDTH
    $DEPTH
```
and evaluate with
```
python3 scripts/traffic_eval.py 
    $RESULTS_PATH
    local_first,accel,headway 
    1,1,0.1
    vel,accel,headway
    1,1,0.1
```